<a href="https://colab.research.google.com/github/mckolu/NodeJS-management-system/blob/main/3_Breast_Cancer_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breast Cancer Detection

**Dataset:** https://github.com/jbrownlee/Datasets/blob/master/breast-cancer.csv

Main source: https://github.com/jbrownlee/Datasets

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/breast-cancer.csv', header=None, names=['age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat', 'rec'])
df.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,rec
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


In [ ]:
df['deg-malig'] = df['deg-malig'].str.replace(r'\D', '').astype(int)

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,rec
0,'40-49','premeno','15-19','0-2','yes',3,'right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no',1,'right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no',2,'left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes',3,'right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes',2,'left','right_up','no','recurrence-events'


In [ ]:
# rec = df[df.rec==1]
# df.groupby('rec').count()


In [ ]:
df['irradiat'] = pd.get_dummies(df['irradiat'], drop_first=True)
df['breast'] = pd.get_dummies(df['breast'], drop_first=True)
df['rec'] = pd.get_dummies(df['rec'], drop_first=True)
df['node-caps'] = pd.get_dummies(df['node-caps'], drop_first=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['age'] = labelencoder.fit_transform(df['age'])

df['tumor-size'] = labelencoder.fit_transform(df['tumor-size'])
df['inv-nodes'] = labelencoder.fit_transform(df['inv-nodes'])

df.head()


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,rec
0,2,'premeno',2,0,1,3,1,'left_up',0,1
1,3,'ge40',2,0,0,1,1,'central',0,0
2,3,'ge40',6,0,0,2,0,'left_low',0,1
3,2,'premeno',6,0,1,3,1,'left_low',1,0
4,2,'premeno',5,4,1,2,0,'right_up',0,1


In [ ]:
# Preprocessing

from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse =False, handle_unknown='ignore') 
converted = pd.DataFrame(encoder.fit_transform(df[['menopause']]), columns=['lt40', 'premeno']) 

converted

df_new = pd.concat([df, converted],axis='columns')
# df_new = df.drop(columns=['menopause'], axis=1)

df_new.head()


,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,rec,lt40,premeno
0,2,'premeno',2,0,1,3,1,'left_up',0,1,0.0,1.0
1,3,'ge40',2,0,0,1,1,'central',0,0,0.0,0.0
2,3,'ge40',6,0,0,2,0,'left_low',0,1,0.0,0.0
3,2,'premeno',6,0,1,3,1,'left_low',1,0,0.0,1.0
4,2,'premeno',5,4,1,2,0,'right_up',0,1,0.0,1.0


In [ ]:
X = df_new.drop(columns=['breast-quad', 'rec', 'menopause'], axis=1)
y = df_new['rec']

X.head()

,age,tumor-size,inv-nodes,node-caps,deg-malig,breast,irradiat,lt40,premeno
0,2,2,0,1,3,1,0,0.0,1.0
1,3,2,0,0,1,1,0,0.0,0.0
2,3,6,0,0,2,0,0,0.0,0.0
3,2,6,0,1,3,1,1,0.0,1.0
4,2,5,4,1,2,0,0,0.0,1.0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # random_state makes sure to generate the same numbers (randomization)len()

logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train, y_train)

y_predicted = logreg.predict(X_test)

logreg.score(X_test, y_test)

# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, y_predicted)
# confusion_matrix(y_test, y_predicted)

0.7931034482758621

# Basics

In [ ]:
import sys

sys.exit()

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# Tesla T4
# !nvidia-smi
!nvidia-smi -L

# https://www.youtube.com/watch?v=8jazNUpO3lQ&list=PLeo1K3hjS3us_ELKYSj_Fth2tIEkdKXvV&index=41
# https://github.com/codebasics/py/blob/master/ML/1_linear_reg/1_linear_regression.ipynb

In [ ]:
import pandas as pd 

student_data = [
    {'name': 'late kalas', 'grade': 2, 'major': 'CS' },
    {'name': 'kate nope', 'grade': 1, 'major': 'science' },
    {'name': 'mate hmm', 'grade': 3, 'major': 'social' },
]

df = pd.DataFrame(data=student_data, columns=['name','grade','major'])

df[['name','major']][df['grade']==df['grade'].max()]


# Linear Regression

* Modeling the relationship as a linear equation through recomputing the weights for each feature. 
* Find the relationship between dependent and independent variables. **y = mx+b** m is the coefficient (slope, gradient) and b is the intercept.


In [ ]:
# Create/Import Data and Visualize

homeprices = [
    {'area': '2600', 'price': 550000},
    {'area': '3000', 'price': 565000},
    {'area': '3200', 'price': 610000},
    {'area': '3600', 'price': 680000},
    {'area': '4000', 'price': 725000}
]

df = pd.DataFrame(data=homeprices, columns=['area','price'])

import matplotlib.pyplot as plt
plt.scatter(df['area'], df['price'], color='blue', marker='+')
plt.xlabel('area square ft')
plt.ylabel('price $US')

* Steps: Square the individual errors, sum them up, and try to minimize the error by taking the derivative of the error iteratively.
* INPUT (independent variables) needs to be 2D (df[[''area]]) and the dependent variable needs to be 1D (df['price']). Predict needs to be the same shape as the input.
* reg.coef_ and reg.intercept_ provide coefficient adn the intercept of the model


In [ ]:
from sklearn import linear_model

lin_reg = linear_model.LinearRegression()

area = df[['area']].values
price = df['price']

lin_reg.fit(area, price)
# reg.coef_
# reg.intercept_

lin_reg.predict([[3300]])


plt.xlabel('area square ft')
plt.ylabel('price $US')
plt.scatter(df['area'], df['price'], color='blue', marker='+')

In [ ]:
# Exercise
data = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/1_linear_reg/Exercise/canada_per_capita_income.csv')

lin = linear_model.LinearRegression()

X = data.iloc[:,0:1]
y = data.iloc[:,-1]

lin.fit(X,y)
lin.predict([[2020]])



# Multivariate Linear Regression

In [ ]:
# The same idea

df2 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/2_linear_reg_multivariate/homeprices.csv')

df2.bedrooms = df2.bedrooms.fillna(df2.bedrooms.median())

X = df2.iloc[:,:-1]
y = df2.iloc[:,-1]

mv_lin = linear_model.LinearRegression()

mv_lin.fit(X,y)

# mv_lin.coef_
# mv_lin.intercept_

mv_lin.predict([[3000,3,40]]) # 498408.25158031
# mv_lin.predict([[2500,4,5]]) # 578876.03748933


In [ ]:
# Exercise:

df3 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/2_linear_reg_multivariate/Exercise/hiring.csv')

# Pre-processing
import math
df3['test_score(out of 10)'] = df3['test_score(out of 10)'].fillna(math.floor(df3['test_score(out of 10)'].mean()))

df3['experience'] = df3['experience'].fillna('zero')

!pip install word2number
from word2number import w2n
df3['experience'] = df3['experience'].apply(w2n.word_to_num)

df3

X = (df3.iloc[:,:-1])
X
y = df3.iloc[:,-1]

mv2_lin = linear_model.LinearRegression()

mv2_lin.fit(X,y)

mv2_lin.predict(X) 

# mv2_lin.predict([[12,10,10]]) 

# Feature Engineering: Converting the categorical variables

## Dummy Variables & One Hot Encoding

There are some cases where LabelEncoder or DictVectorizor are useful, but these are quite limited in my opinion due to ordinality.

LabelEncoder can turn [dog,cat,dog,mouse,cat] into [1,2,1,3,2], but then the *imposed* ordinality means that the average of dog and mouse is cat. Still there are algorithms like decision trees and random forests that can work with categorical variables just fine and LabelEncoder can be used to store values using less disk space.

One-Hot-Encoding has the advantage that the result is binary rather than ordinal and that everything sits in an orthogonal vector space. The disadvantage is that for high cardinality, the feature space can really blow up quickly and you start fighting with the curse of dimensionality. In these cases, I typically employ one-hot-encoding followed by PCA for dimensionality reduction. I find that the judicious combination of one-hot plus PCA can seldom be beat by other encoding schemes. PCA finds the linear overlap, so will naturally tend to group similar features into the same feature.



In [ ]:
# One hot encoding, converts the categorical features to a matrix. However, we need to drop the first column of the matrix to avoid milticollinearity (dummy variable trap).
# Modern machine learning algorithms, though, don’t require their inputs to be linearly independent and use methods such as L1 regularization to prune redundant inputs

df = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/5_one_hot_encoding/homeprices.csv')
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

from sklearn.preprocessing import OneHotEncoder
# drop='if_binary': drop a column for feature only having 2 categories
# drop='first': Drop the first column to prevent collinearity

encoder = OneHotEncoder(drop='first', sparse =False, handle_unknown='ignore') 
converted = pd.DataFrame(encoder.fit_transform(X[['town']]), columns=['town_r', 'town_ww']) 

X = pd.concat([X, converted],axis='columns')
X = X.drop(columns=['town'], axis=1)

mv_lin2 = linear_model.LinearRegression()
mv_lin2.fit(X,y)

mv_lin2.predict([[3100,0,0]]) # 498408.25158031

mv_lin2.score(X,y)


In [ ]:
# Exercise: https://github.com/codebasics/py/blob/master/ML/5_one_hot_encoding/Exercise/exercise_one_hot_encoding.ipynb

df5 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/5_one_hot_encoding/Exercise/carprices.csv')
encoder = OneHotEncoder(sparse =False, handle_unknown='ignore') 
dummy = pd.DataFrame(encoder.fit_transform(df5[['Car Model']]), columns=['Audi A5', 'BMW X5', 'Mercedez']) 

df5 = pd.concat([df5, dummy], axis =1)
# df5
X = df5.drop(columns =['Car Model', 'Audi A5', 'Sell Price($)'])
y = df5['Sell Price($)']

mv_lin4 = linear_model.LinearRegression()
mv_lin4.fit(X,y)

mv_lin4.predict([[86000,7, 1,0]]) 

mv_lin4.score(X,y)



# Splitting Training and Testing Data

In [ ]:
df7 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/6_train_test_split/carprices.csv')
# plt.scatter(df7['Mileage'], df7['Sell Price($)'])

X = df7[['Mileage','Age(yrs)']]
y = df7['Sell Price($)']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=10) # random_state makes sure to generate the same numbers (randomization)
# X_train

In [ ]:
from sklearn.linear_model import LinearRegression
clf = LinearRegression()
clf.fit(X_train, y_train)

clf.predict(X_test)

clf.score(X_test, y_test)

# Logistic Regression
Logistic regression is just one step away from reaching Logistic Regression: The activation function(Sigmoid or softmax)

In logistic regression, we decide a probability threshold. If the probability of a particular element is higher than the probability threshold (usually 0.5 for binary classification) then we classify that element in one group or vice versa.

The sigmoid function is used for the two-class logistic regression, whereas the softmax function is used for the multiclass logistic regression (a.k.a. MaxEnt, multinomial logistic regression, softmax Regression, Maximum Entropy Classifier


In [ ]:
df8 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/7_logistic_reg/insurance_data.csv')
df8.head()



In [ ]:
plt.scatter(df8['age'], df8['bought_insurance'])

In [ ]:
X = df8[['age']]
y = df8['bought_insurance']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=10) # random_state makes sure to generate the same numbers (randomization)
X_test

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

log_reg.predict(X_test)

log_reg.score(X_test, y_test)

In [ ]:
log_reg.predict_proba(X_test) # probability for each data point

In [ ]:
log_reg.predict([[77]])
log_reg.predict_proba([[77]])


In [ ]:
# Exercise
# https://github.com/codebasics/py/blob/master/ML/7_logistic_reg/Exercise/7_logistic_regression_exercise.ipynb

df0 = pd.read_csv('https://raw.githubusercontent.com/codebasics/py/master/ML/7_logistic_reg/Exercise/HR_comma_sep.csv')
df0.head()

In [ ]:
# Exploratory Data Analysis

df0.groupby('left').mean()

In [ ]:
pd.crosstab(df0.salary,df0.left).plot(kind='bar')

In [ ]:
df0['Department'].unique()

from sklearn.preprocessing import OneHotEncoder

# sparse matrix: A matrix that has been compressed to eliminate zero-values.,
# Dense matrix: bunch of zeros.
# sparse matrices are better for memory and time complexity in general

encoder1 = OneHotEncoder(drop='first', sparse =False, handle_unknown='ignore')  
converted1 = pd.DataFrame(encoder1.fit_transform(df0[['Department']])) 
converted2 = pd.DataFrame(encoder1.fit_transform(df0[['salary']])) 


X = pd.concat([df0, converted1, converted2],axis='columns')
X = X.drop(columns=['left', 'Department', 'salary'], axis=1)
y = df0['left']


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=10) # random_state makes sure to generate the same numbers (randomization)

log_reg2 = LogisticRegression(max_iter=1000)
log_reg2.fit(X_train, y_train)

log_reg2.predict(X_test)

log_reg2.score(X_test, y_test)

#Multiclass Logistic Regression

In [ ]:
from sklearn.datasets import load_digits
%matplotlib inline
import matplotlib.pyplot as plt
digits = load_digits()

In [ ]:
dir(digits)

In [ ]:
(digits.data[0]).shape

# (digits.data[0]).shape

digits.target_names

In [ ]:
plt.gray()
plt.matshow(digits.images[0])

# plt.gray() 
# for i in range(5):
#     plt.matshow(digits.images[i]) 

In [ ]:
digits.target[0].shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(digits.data,digits.target, test_size=0.2, random_state=10) # random_state makes sure to generate the same numbers (randomization)len()
len(X_train)

In [ ]:
log_reg3 = LogisticRegression(max_iter=10000)
log_reg3.fit(X_train, y_train)

# log_reg3.predict(X_test)

log_reg3.score(X_test, y_test)

In [ ]:
plt.matshow(digits.images[67])

In [ ]:
digits.target[67]
log_reg3.predict(digits.data[0:5])

In [ ]:
# confusion matrix

y_predicted = log_reg3.predict(X_test)


from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)


confusion_matrix(y_test, y_predicted)


In [ ]:
# Exercise

from sklearn.datasets import load_iris
%matplotlib inline
import matplotlib.pyplot as plt

iris = load_iris()

dir(iris)

X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

iris_logreg = LogisticRegression(max_iter=10000)
iris_logreg.fit(X_train, y_train)

y_predicted = iris_logreg.predict(X_test)

iris_logreg.score(X_test, y_test)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)

confusion_matrix(y_test, y_predicted)

In [ ]:
# Exercise 2
# https://www.digitalocean.com/community/tutorials/mnist-dataset-in-python

from keras.datasets import mnist
data = mnist.load_data()
(train_X, train_y), (test_X, test_y) = mnist.load_data()

# dir(mnist)

train_X.shape

In [ ]:
from matplotlib import pyplot

pyplot.imshow(train_X[5], cmap=pyplot.get_cmap('gray'))
pyplot.show()


# More preprocessing

https://scikit-learn.org/stable/modules/preprocessing.html

# Save a Model with Pickle

In [ ]:
# import pickle

# with open('model_pickle','wb') as file:
#     pickle.dump(mv2_lin,file)

# with open('model_pickle','rb') as file:
#     mp = pickle.load(file)

# mp.predict([[12,10,10]]) 

# Save for later